In [302]:
###imports
import cv2
import numpy as np

<br>

Task 1

<br>

In [38]:
def find_image_coordinates(img, image_path):
    

    # Display the image and allow user to select 5 points
    points = []

    def select_point(event, x, y, flags, param):
        # Register the mouse callback function
        if event == cv2.EVENT_LBUTTONDOWN and len(points) < 5:
            # Append the image coordinates of the selected point to the list
            points.append((x, y))
            cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
            cv2.putText(img, str(len(points)), (x + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.imshow("Image", img)

    cv2.namedWindow("Image")
    cv2.setMouseCallback("Image", select_point)

    while True:
        cv2.imshow("Image", img)
        if cv2.waitKey(0) or len(points) == 5:  # Press Esc or select 10 points to exit
            break

    cv2.destroyAllWindows()
    return points

In [39]:
## for path to iindividual images
image_path1 = "house1.png"  
image_path2 = "house2.png"  

In [40]:
# Load the image and read the world coordinates

img1 = cv2.imread('house1.png')
img2 = cv2.imread('house2.png')

In [41]:
##get the points of image 1

image_points1 = find_image_coordinates(img1, image_path1)
image_points1

[(234, 409), (375, 342), (415, 432), (638, 346), (572, 247)]

In [42]:
##get the points of image 2
img2 = cv2.imread('house2.png')
image_points2 = find_image_coordinates(img2, image_path2)
image_points2

[(238, 440), (325, 371), (343, 473), (572, 395), (422, 254)]

8 Point Algorithm
------------------------

0. (Normalize points)
1. Construct the M x 9 matrix A
2. Find the SVD of ATA
3. Entries of F are the elements of column of
V corresponding to the least singular value
4. (Enforce rank 2 constraint on F)
5. (Un-normalize F)

In [291]:

# Convert image points to homogeneous coordinates
image_points1_homo = np.hstack((image_points1, np.ones((len(image_points1), 1))))
image_points2_homo = np.hstack((image_points2, np.ones((len(image_points2), 1))))

# Normalize image points
image_points1_normalized = cv2.normalize(image_points1_homo, None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
image_points2_normalized = cv2.normalize(image_points2_homo, None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)


In [292]:
###construct Mx9 matrix A

A = np.zeros((len(image_points2_normalized), 9))

for i in range(len(image_points2_normalized)-1):
    A[i] = np.concatenate([np.array([0, 0, 0]), - image_points2_normalized[i][2] * image_points1_normalized[i], image_points2_normalized[i][1] * image_points1_normalized[i]])
    A[i + 1] = np.concatenate([image_points2_normalized[i][2] * image_points1_normalized[i], np.array([0, 0, 0]), - image_points2_normalized[i][0] * image_points1_normalized[i]])


In [293]:

_, _, V  = np.linalg.svd(A.T  @ A)
V

array([[-0.19372915, -0.01611875,  0.19372915, -0.2469848 , -0.13775268,
         0.69097449, -0.29540533, -0.09308435,  0.52088979],
       [-0.47510512, -0.03952995,  0.47510512,  0.16150123,  0.10164613,
        -0.53649658, -0.41202619,  0.01564395,  0.22963919],
       [ 0.04681742,  0.00389533, -0.04681742, -0.87301262,  0.18259416,
        -0.17850254, -0.30652009,  0.11468388, -0.24726365],
       [-0.24909889, -0.02072566,  0.24909889,  0.16288213,  0.58196445,
         0.40953281,  0.04700054,  0.42966277, -0.39461432],
       [-0.19172305, -0.01595184,  0.19172305, -0.32104047,  0.02538071,
        -0.15244813,  0.75132804,  0.28636067,  0.39087368],
       [-0.66136308, -0.08038454, -0.7281802 ,  0.00243741,  0.12805942,
        -0.00975881,  0.00700762, -0.07505675,  0.06089163],
       [ 0.25964507, -0.84746739, -0.14544798,  0.07259809,  0.07358659,
        -0.04843205, -0.13905332,  0.3331971 ,  0.22325114],
       [-0.26981007, -0.52201661,  0.29037382, -0.12567528, -0

In [294]:
##the fundamental matrix 
F = V[: ,8].reshape(3, 3) 
F

array([[ 0.52088979,  0.22963919, -0.24726365],
       [-0.39461432,  0.39087368,  0.06089163],
       [ 0.22325114, -0.41313933, -0.28646936]])

In [295]:
ePrime

array([[ 0.42606107],
       [-0.06792765],
       [ 0.90214068]])

In [296]:
##enforcing the rank 2 constraint on F
s, v, d = np.linalg.svd(F)
F =  s @ np.diag([*v[:2], 0]) @ d
F

array([[ 0.52801975,  0.22850245, -0.2321667 ],
       [-0.33507309,  0.38138092,  0.18696409],
       [ 0.28087737, -0.42232678, -0.16445171]])

In [297]:
#denormalise matrix
# F = image_points2_homo.T @ F @ image_points1_homo
# F

<br>

Task 2

<br>

In [298]:
##got the value from the calculation of the s,d above
e = d[-1]
e

array([-0.42606107,  0.06792765, -0.90214068])

In [299]:
###epipole from fundamental meatrix
ePrime = cv2.normalize(e, None, alpha=-1)
ePrime

array([[ 0.42606107],
       [-0.06792765],
       [ 0.90214068]])

In [303]:
#### linear triangulation method using the DLT as described in
##Sec. 12.2 in the book Multiple View Geometry

num_points = len(image_points1)
points_3d = np.zeros((num_points, 3))

##canonical camera pair p and p' 
P = np.zeros(len(image_points2_normalized))
P = np.eye(3)
P_prime = ePrime.T @ F* ePrime


for i in range(num_points):
    x1, y1 = image_points1[i]
    x2, y2 = image_points2[i]

    # Homogeneous image coordinates
    x1_homo = np.array([[x1], [y1], [1]])
    x2_homo = np.array([[x2], [y2], [1]])

    # Linear triangulation - method from the book
    A = np.zeros((len(image_points2_normalized), 3))
    A[0] = x1_homo[0] * P[2] - P[0]
    A[1] = x1_homo[1] * P[2] - P[1]
    A[2] = x2_homo[0] * P_prime[2] - P_prime[0]
    A[3] = x2_homo[1] * P_prime[2] - P_prime[1]

    _, _, V = np.linalg.svd(A)
    X_homo = V[-1]
    X_homo /= X_homo[2]  # Normalize by the fourth coordinate

    # Convert homogeneous coordinates to 3D coordinates
    X = X_homo[:3]

    # Store the 3D point
    points_3d[i] = X


In [304]:
#reconstucted 3d points
points_3d

array([[248.1671504 , 400.91217912,   1.        ],
       [277.62311986, 448.59342055,   1.        ],
       [322.30099415, 520.92268426,   1.        ],
       [439.76245164, 711.07554803,   1.        ],
       [399.48588637, 645.86793935,   1.        ]])

<br>

Task 3

<br>

In [305]:
world_coords = np.loadtxt('coords.txt')

In [306]:
###cordinates x in coords.tex

for X, Y, _, _ in world_coords:
    print(X)

-1.0
0.015545
0.491684
0.0
1.05319
1.22323
0.0
1.0
-1.05319
1.0
